# Applications Processing Automation

<a target="_blank" href="https://colab.research.google.com/github/trendinafrica/student_selection_process_automation/blob/main/main.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

(*By: [@mahmoud-elmakki](https://github.com/mahmoud-elmakki)*)

The purpose of this code is to automate the first trivial filtering steps in the processing of the applications for the TReND in Africa Computational Neuroscience and Machine Learning Basics course.

This code is organized as a set of functions to be applied as a processing pipeline on the application responses data (See [documentation](https://docs.google.com/document/d/1n4pMEOgMuenuFpN6zXQtZlpYFXwPat2P4-SzZaN8mFg/edit?usp=drivesdk)).

Also see [weighted_grading.ipynb](https://github.com/trendinafrica/student_selection_process_automation/blob/main/weighted_grading.ipynb)

### **How to use (as a developer):**
Just clone the Github repository and get into the business!\
If you have anaconda and yupyter installed locally you can just clone the repory directly on your machine. Elsewise, you can clone it into Google Colab.
(In either case, don't forget to regularly pull and push changes).

### **How to use (as a reviewer):**
If you are on Github now, open this notebook in Google Colab, or clone the whole repo locally, so you can run the cells. In case of running it in Colab, don't forget to save and download the resulting Excel sheet of the processed responses into a local folder.

In [730]:
import numpy as np
import pandas as pd

import os

In [731]:
# Note that you have to download the responses data Excel sheet from Google Drive and put it in the same folder as the code.
# You don't have to do this if you cloned the Github repo (all will be organized in the repo).
 
# TODO: Load data directly from Google Drive.

# Loading students responses data
STD_DATA_DIR = './responses_data/TReND Comp Neuro application form Rwanda 2024 (Responses).xlsx'
std_raw_responses_df = pd.read_excel(STD_DATA_DIR)

# Loading references responses data
REF_DATA_DIR = './responses_data/Recommendation Letter Portal (Responses).xlsx'
ref_raw_responses_df = pd.read_excel(REF_DATA_DIR)

# Adding two columns to the responses DataFrame (initialized with None for all cells).
std_raw_responses_df['Flag'] = None  #String ("flagged" or None)
std_raw_responses_df['Notes'] = None #String (Text of notes == reasaons for flagging)
std_raw_responses_df['Recommendation Letter 1'] = None
std_raw_responses_df['Recommendation Letter 2'] = None

# Flag the reference response if they submit more than one letter (keeo the last letter submitted)
ref_raw_responses_df['Flag'] = None
ref_raw_responses_df['Notes'] = None
ref_raw_responses_df['Matched'] = "unmatched"

# Just specify folder names - thje code will create the directory
RESULTS_FOLDER_NAME = "filtered_responses"
RESULTS_DIR = os.path.join(os.getcwd(), RESULTS_FOLDER_NAME)

if not os.path.exists(RESULTS_DIR):
    os.mkdir(RESULTS_DIR)
    
LETTERS_STATS_FOLDER_NAME = "letters_stats"
LETTERS_STATS_DIR = os.path.join(os.getcwd(), LETTERS_STATS_FOLDER_NAME)

if not os.path.exists(LETTERS_STATS_DIR):
    os.mkdir(LETTERS_STATS_DIR)

In [732]:
# Specify the minimum and maximun number of words for ansewrs for essay questions.
# Note: These parameters apply for for all essay questions.

MIN_WORDS_NUM = 10
MAX_WORDS_NUM = 500

UNOFFICIAL_EMAILS = ["gmail", "yahoo", "hotmail"]

In [733]:
# Use this dictionary as a reference for column names.

std_questions_dict = {i: column for i, column in enumerate(std_raw_responses_df.columns)}
std_questions_dict

{0: 'Timestamp',
 1: 'Email address',
 2: 'First Name',
 3: 'Last Name',
 4: 'Age',
 5: 'Nationality',
 6: 'In case you are selected, where would you be traveling from to Rwanda?',
 7: 'Gender',
 8: 'What is your highest degree of education?',
 9: 'What is your current career stage?',
 10: 'Which group of studies best describes your background?',
 11: 'Name of current University/Research Institution/Organization (e.g. Department of Biomedical Engineering, CMU-Africa, Kigali, Rwanda)',
 12: 'Field of studies in Undergraduate (completed or ongoing, eg. Neuroscience, Mathematics, Law)',
 13: 'Field of studies in Master’s (if applicable, completed or ongoing, eg. Neuroscience, Mathematics, Law)',
 14: 'Field of studies in PhD/MD degree (if applicable, completed or ongoing, eg. Neuroscience, Mathematics, Law)',
 15: 'Current research focus or research focus of the last research project you were engaged in (if applicable)',
 16: 'Rate your proficiency of computer programming (with any progra

In [734]:
# Use this dictionary as a reference for column names.

ref_questions_dict = {i: column for i, column in enumerate(ref_raw_responses_df.columns)}
ref_questions_dict

{0: 'Timestamp',
 1: 'Email address',
 2: 'Please enter your title, name and affiliation.',
 3: 'Please enter the first name of the student you are supporting.',
 4: 'Please enter the last name of the student you are supporting.',
 5: 'Please enter the email address of the student you are supporting.',
 6: 'Please enter your relationship to the student.',
 7: 'Please upload your recommendation letter in pdf format.  If you have trouble attaching the file, please send it to compneuroai@trendinafrica.org through your institutional email address with the name: StudentName_StudentSurname_ReccomentationLetter.pdf',
 8: 'Flag',
 9: 'Notes',
 10: 'Matched'}

In [735]:
# Used indices of the student responses DataFrame

std_idcs = {
    'email_idx' : 1,
    'firstname_idx' : 2,
    'lastname_idx' : 3,
    'ref' : {
        'first_ref_email_idx' : 25,
        'second_ref_email_idx' : 27
          },
    'flag_idx' : 28,
    'notes_idx' : 29,
    'first_recomm_letter_idx' : 30,
    'second_recomm_letter_idx' : 31,
}

# Used indices of the reference responses DataFrame
ref_idcs = {
    'email_idx' : 1,
    'name_idx' : 2,
    'std' : {
        'firstname_idx' : 3,
        'lastname_idx' : 4,
        'email_idx' : 5
    },
    'letter_idx' : 7,
    'flag_idx' : 8,
    'notes_idx' : 9,
    'matched_idx' : 10
}

std_str_qs = [std_idcs['email_idx'], std_idcs['firstname_idx'], std_idcs['lastname_idx']]
ref_str_qs = [ref_idcs['email_idx'], ref_idcs['std']['email_idx'], ref_idcs['std']['firstname_idx'], ref_idcs['std']['lastname_idx']]

std_names = [std_idcs['firstname_idx'], std_idcs['lastname_idx']]
ref_names = [ref_idcs['std']['firstname_idx'], ref_idcs['std']['lastname_idx']]

# Carefully specify names of the columns to be processed (mostly responses for essay questions).
essay_qs = [20, 21, 22]

## Utility functions

In [736]:
def word_count(answer):
    """
    Takes a specific answer (cell) of a specific essay question and returns the answer's number of words.
    """
    return len(answer.split())


def to_lowercase(std_df, ref_df, std_str_qs=std_str_qs, ref_str_qs=ref_str_qs):
    """
    For more rigid string comparisons, convert all answers needed for comparison to lowercase.
    """
    for q in std_str_qs:
        std_df[q] = std_df[q].str.lower()
        
    for q in ref_str_qs:
        ref_df[q] = ref_df[q].str.lower()
        
    return std_df, ref_df


def to_uppercase(std_df, ref_df, std_names=std_names, ref_names=ref_names):
    """
    This to bring names back as they were.
    """
    for q in std_names:
        std_df[q] = std_df[q].str.title()
        
    for q in ref_names:
        ref_df[q] = ref_df[q].str.title()
        
    return std_df, ref_df


def remove_spaces(std_df, std_str_qs, ref_df, ref_str_qs):
    """
    Remove spaces from names and emails.
    """
    for q in std_str_qs:
        std_df[q] = std_df[q].str.replace(" ", "")
        
    for q in ref_str_qs:
        ref_df[q] = ref_df[q].str.replace(" ", "")
        
    return std_df, ref_df


def set_flag(responses_df, email):
    """
    Sets the 'flag' column value to "flagged" for a response chosen by it's 'Email address'
    """
    # This modifies the DataFrame itself (i.e change in place)
    responses_df.iloc[responses_df[std_idcs['email_idx']] == email, std_idcs['flag_idx']] = "flagged"

    
def leave_note(responses_df, response_index, note_text):
    """
    Appends a note to the 'Notes' column.
    """
    edited_responses_df = responses_df.copy()
    
    if note_text not in str(edited_responses_df.iloc[response_index, std_idcs['notes_idx']]):
        if edited_responses_df.iloc[response_index, std_idcs['notes_idx']] is None:
            edited_responses_df.iloc[response_index, std_idcs['notes_idx']] = note_text
        else:
            edited_responses_df.iloc[response_index, std_idcs['notes_idx']] = str(edited_responses_df.iloc[response_index, std_idcs['notes_idx']]) + ". " + note_text 
            
    return edited_responses_df


def leave_note_for_ref(ref_responses_df, response_index, note_text):
    """
    Appends a note to the 'Notes' column.
    """
    edited_ref_responses_df = ref_responses_df.copy()
    
    if note_text not in str(ref_responses_df.iloc[response_index, ref_idcs['notes_idx']]):
        if edited_ref_responses_df.iloc[response_index, ref_idcs['notes_idx']] is None:
            edited_ref_responses_df.iloc[response_index, ref_idcs['notes_idx']] = note_text
        else:
            edited_ref_responses_df.iloc[response_index, ref_idcs['notes_idx']] = str(edited_ref_responses_df.iloc[response_index, ref_idcs['notes_idx']]) + ". " + note_text 
            
    return edited_ref_responses_df

    
def column_names_to_indices(df, indices_dict):
    """
    Replaces column names with indices.
    """
    processed_df = df.rename(columns={column: i for i, column in enumerate(indices_dict.values())})

    return processed_df


def indices_to_column_names(df, indices_dict):
    """
    Replaces indices with column names.
    """
    processed_df = df.rename(columns={i: column for i, column in enumerate(indices_dict.values())})

    return processed_df


def remove_flagged(df):
    """
    Remove f;agged columns.
    """
    processed_df = df.drop(df[(df[std_idcs['flag_idx']] == 'flagged')].index)
    
    return processed_df


def get_unmatched_letters(std_responses_df, ref_responses_df):
    """
    Gets unmatched recommendation letters.
    """
    ref_responses_df_unmatched = ref_responses_df.loc[ref_responses_df[ref_idcs['matched_idx']] == "unmatched"]

    return ref_responses_df_unmatched


def got_not_enough_letters(std_responses_df):

    return std_responses_df.loc[(std_responses_df[std_idcs['first_recomm_letter_idx']].isnull()) | (std_responses_df[std_idcs['second_recomm_letter_idx']].isnull())] 


def got_enough_letters(std_responses_df):

    return std_responses_df.loc[(std_responses_df[std_idcs['first_recomm_letter_idx']].notnull()) & (std_responses_df[std_idcs['second_recomm_letter_idx']].notnull())] 


def got_no_letters(std_responses_df):

    return std_responses_df.loc[(std_responses_df[std_idcs['first_recomm_letter_idx']].isnull()) & (std_responses_df[std_idcs['second_recomm_letter_idx']].isnull())] 


def get_letters_counts(ref_responses_df):
    
    return ref_responses_df.groupby([ref_idcs['std']['email_idx']]).size().reset_index().rename(columns={0: '# letters'})


def save_letters_stats(std_responses_df, ref_responses_df, letters_stats_dir=LETTERS_STATS_DIR):
    
    got_enough_letters(std_responses_df).sort_values(by=[std_idcs['firstname_idx']]).to_excel(letters_stats_dir + "/got_enough_letters.xlsx")
    got_not_enough_letters(std_responses_df).sort_values(by=[std_idcs['firstname_idx']]).to_excel(letters_stats_dir + "/got_not_enough_letters.xlsx")
    got_no_letters(std_responses_df).sort_values(by=[std_idcs['firstname_idx']]).to_excel(letters_stats_dir + "/got_no_letters.xlsx")
    
    get_letters_counts(ref_responses_df).to_excel(letters_stats_dir + "/letters_counts(1 or 2).xlsx")


def get_std_by_email(std_responses_df, email):
    
    return std_responses_df.loc[std_responses_df[std_idcs['email_idx']] == email]


def get_ref_by_email(ref_responses_df, email):
    
    return ref_responses_df.loc[ref_responses_df[ref_idcs['email_idx']] == email]


def get_std_by_email_from_ref(ref_responses_df, std_email):
    
    return ref_responses_df.loc[ref_responses_df[ref_idcs['std']['email_idx']] == std_email]


def get_std_by_firstname_from_ref(ref_responses_df, firstname):
    
    return ref_responses_df.loc[ref_responses_df[ref_idcs['std']['firstname_idx']] == firstname]


def get_std_by_lastname_from_ref(ref_responses_df, lastname):
    
    return ref_responses_df.loc[ref_responses_df[ref_idcs['std']['lastname_idx']] == lastname]


def get_std_by_firstname_and_lastname_from_ref(ref_responses_df, std_firstname, std_lastname):
    
    return ref_responses_df.loc[(ref_responses_df[ref_idcs['std']['firstname_idx']] == std_firstname) & (ref_responses_df[ref_idcs['std']['lastname_idx']] == std_lastname)]


def get_std_emails(std_responses_df):
    
    return std_responses_df.iloc[:, std_idcs['email_idx']:std_idcs['email_idx'] + 1]


def get_ref_emails(ref_responses_df):
    
    return ref_responses_df.iloc[:, ref_idcs['email_idx']:ref_idcs['email_idx'] + 1]


def get_std_names(std_responses_df):

    return std_responses_df.iloc[:, std_idcs['firstname_idx']:std_idcs['lastname_idx'] + 1]


def get_ref_names(ref_responses_df):

    return ref_responses_df.iloc[:, ref_idcs['name_idx']:std_idcs['name_idx'] + 1]


def get_std_summary(std_responses_df):
    
    return pd.concat([std_responses_df.iloc[:, std_idcs['email_idx']:std_idcs['firstname_idx']+2], std_responses_df.iloc[:, std_idcs['flag_idx']:]], axis=1)

## Main Pipeline

In [760]:
def remove_duplicates(responses_df):
    """
    removes duplicated rows (responses) based on 'Email address' and keeps the last response submitted.
    Note: Some students may make changes to their responses and submit a new one,
    this's why this function keeps the last response submitted and removes preceding ones.
    
    TODO: Check with the organizers what else is an adequate action.
    
    params :
        response_df: the responses data (DataFrame)
    returns:
        edited_responses_df: An edited response_df with duplicates removed
    """
    
    edited_responses_df = responses_df.copy()
    
    edited_responses_df.drop_duplicates(subset=[std_idcs['email_idx']], keep='last')
    
    return edited_responses_df


def flag_duplicates(responses_df):
    """
    flags duplicated rows (responses) based on 'Email address' and keeps the last response submitted.
    Note: Some students may make changes to their responses and submit a new one,
    this's why this function keeps the last response submitted and flag preceding ones, and leaves a note.
    
    TODO: Check with the organizers what else is an adequate action.
    
    params :
        response_df: the responses data (DataFrame)
    returns:
        edited_responses_df: An edited responses_df with 'flag' column updated
    """
    
    edited_responses_df = responses_df.copy()
    
    # Format: df['col'] = (value_if_false).where(condition, value_if_true)
    
    edited_responses_df[std_idcs['flag_idx']] = (edited_responses_df[std_idcs['flag_idx']]).where(
        # True/False nupmy array - True: duplicated, False: unique (before inversion)
        np.invert(np.array(edited_responses_df.duplicated(subset=[1], keep='last'))),
        "flagged"
    )

    edited_responses_df[std_idcs['notes_idx']] = (edited_responses_df[std_idcs['notes_idx']]).where(
        # True/False nupmy array - True: duplicated, False: unique (before inversion)
        np.invert(np.array(edited_responses_df.duplicated(subset=[1], keep='last'))),
        "A duplicated response"
    )
    
    return edited_responses_df


def flag_duplicate_refs(ref_responses_df):
    """
    flags duplicated reference responces (i.e. submitting multiple letters), and keep the last submitted one.
    
    TODO: Check with the organizers what else is an adequate action.
    
    params :
        ref_response_df: the responses data (DataFrame)
    returns:
        edited_ref_responses_df: An edited responses_df with 'flag' column updated
    """
    
    edited_ref_responses_df = ref_responses_df.copy()
    
    # Format: df['col'] = (value_if_false).where(condition, value_if_true)
    
    edited_ref_responses_df[ref_idcs['flag_idx']] = (edited_ref_responses_df[ref_idcs['flag_idx']]).where(
        # True/False nupmy array - True: duplicated, False: unique (before inversion)
        np.invert(np.array(edited_ref_responses_df.duplicated(subset=[ref_idcs['email_idx'], ref_idcs['std']['email_idx']], keep='last'))),
        "flagged"
    )

    edited_ref_responses_df[ref_idcs['notes_idx']] = (edited_ref_responses_df[ref_idcs['notes_idx']]).where(
        # True/False nupmy array - True: duplicated, False: unique (before inversion)
        np.invert(np.array(edited_ref_responses_df.duplicated(subset=[ref_idcs['email_idx'], ref_idcs['std']['email_idx']], keep='last'))),
        "Submitted more than one letter for the same student"
    )
    
    return edited_ref_responses_df


def flag_refs_with_no_students(std_responses_df, ref_responses_df):
    """
    Flag referees whose students didn't submit an application.
    """
    
    edited_ref_responses_df = ref_responses_df.copy()
    
    for ref_idx in range(len(edited_ref_responses_df)):
        
        if edited_ref_responses_df.iloc[ref_idx, ref_idcs['std']['email_idx']] not in std_responses_df[std_idcs['email_idx']].tolist():
            
            #edited_ref_responses_df.iloc[ref_idx, ref_idcs['flag_idx']] = "flagged"
            edited_ref_responses_df = leave_note_for_ref(edited_ref_responses_df, ref_idx, "Their student didn't submit an application, or they misspelled the studen't email address")  
            

    return edited_ref_responses_df


def flag_short(responses_df, essay_qs):
    """
    flags insufficently short answers (less than a specific lower limit) for a specified
    set of essay questions, and leaves a note.
    
    params :
        response_df: the responses data (DataFrame)
        essay_qs   : essay questions (list)
    returns:
        edited_responses_df: An edited responses_df with short answers flagged
    """
    
    edited_responses_df = responses_df.copy()
    
    # Go through all the responses and for each response go through the answers for the essay questions
    for row_index in range(len(edited_responses_df)):
        
        for question in essay_qs:
            
            if word_count(str(edited_responses_df.iloc[row_index, question])) < MIN_WORDS_NUM:
                edited_responses_df.iloc[row_index, std_idcs['flag_idx']] = "flagged"
                
                edited_responses_df = leave_note(edited_responses_df, row_index, "Insufficient short answer/s")        
                    
    return edited_responses_df
                    

# Should we flag long answers ??
def flag_long(responses_df, essay_qs):
    """
    flags extremely long answers (more than a specific upprt limit) for a specified
    set of essay questions, and leaves a note.
    
    params :
        response_df: the responses data (DataFrame)
        essay_qs   : essay questions (list)
    returns:
        edited_response_df: An edited responses_df with long answers flagged
    """
     
    edited_responses_df = responses_df.copy()
    
    # Go through all the responses and for each response go through the answers for the essay questions
    for row_index in range(len(edited_responses_df)):
        for question in essay_qs:
            
            if word_count(str(edited_responses_df.iloc[row_index, question])) > MAX_WORDS_NUM:
                edited_responses_df.iloc[row_index, std_idcs['flag_idx']] = "flagged"
                
                edited_responses_df = leave_note(edited_responses_df, row_index, "Extremely long answer/s")
                        
    return edited_responses_df


def flag_unofficial_emails(responses_df, unofficial_emails=UNOFFICIAL_EMAILS):
    
    edited_responses_df = responses_df.copy()
    
    for row_index in range(len(edited_responses_df)):
        
        try:
            is_unofficial = any([unofficial_email in edited_responses_df.iloc[row_index, std_idcs['ref']['first_ref_email_idx']] for unofficial_email in unofficial_emails] + [unofficial_email in edited_responses_df.iloc[row_index, std_idcs['ref']['second_ref_email_idx']] for unofficial_email in unofficial_emails])
        
        except TypeError:
            is_unofficial = True
        
        if is_unofficial:
        
            edited_responses_df.iloc[row_index, std_idcs['flag_idx']] = "flagged"
            edited_responses_df = leave_note(edited_responses_df, row_index, "Submitted an unofficial email for one of or both of their referees")        
                    
    return edited_responses_df


def match_refs_based_on_stdn_email(std_responses_df, ref_responses_df):
    """
    Matches references with the student/s they are supporting, and flags student response if they get less than the required
    number of reference letters, and leaves a note.
    
    params :
        std_responses_df : students responses data (DataFrame)
        ref_responses_df : references responses data (DataFrame)
    returns:
        edited_std_responses_df: An edited std_responses_df with answers with unsatisfied conditions for recommendation letters flagged
        ref_responses_df: The ref_responses_df but with marking the "Matched" column for letters those successfully matched.
    """
     
    ref_responses_df = flag_duplicate_refs(ref_responses_df)
    edited_std_responses_df = std_responses_df.copy()
    
    for row_index in range(len(edited_std_responses_df)):
        
        if edited_std_responses_df.iloc[row_index, std_idcs['first_recomm_letter_idx']] is not None and edited_std_responses_df.iloc[row_index, std_idcs['second_recomm_letter_idx']] is not None:
            continue
            
        # Flag student response if BOTH of their references didn't submit any letter
        
        if edited_std_responses_df.iloc[row_index, std_idcs['email_idx']] not in ref_responses_df[ref_idcs['std']['email_idx']].values:
            
            edited_std_responses_df.iloc[row_index, std_idcs['flag_idx']] = "flagged"
            edited_std_responses_df = leave_note(edited_std_responses_df, row_index, "Got no recommendation letters")
        
        # Flag student response if ANY of their references didn't submit any letter
        # Assign the one submitted letters to that student
        
        elif ref_responses_df[ref_idcs['std']['email_idx']].value_counts()[edited_std_responses_df.iloc[row_index, std_idcs['email_idx']]] == 1:
            
            edited_std_responses_df.iloc[row_index, std_idcs['flag_idx']] = "flagged"
            edited_std_responses_df = leave_note(edited_std_responses_df, row_index, "Got only one recommendation letter")
            
            for ref_index in range(len(ref_responses_df)):
                
                if ref_responses_df.iloc[ref_index, ref_idcs['std']['email_idx']] == edited_std_responses_df.iloc[row_index, std_idcs['email_idx']]:
                    edited_std_responses_df.iloc[row_index, std_idcs['first_recomm_letter_idx']] = ref_responses_df.iloc[ref_index, ref_idcs['letter_idx']]
                    ref_responses_df.iloc[ref_index, ref_idcs['matched_idx']] = "matched"
        
        # This, from here below, would look much prettier with a while loop!
    
        # If BOTH references subnitted ONLY ONE letter,
        # Assign the right two letters to the specific student
        
        elif ref_responses_df[ref_idcs['std']['email_idx']].value_counts()[edited_std_responses_df.iloc[row_index, std_idcs['email_idx']]] == 2:
            
            for ref_index in range(len(ref_responses_df)):
                
                if ref_responses_df.iloc[ref_index, ref_idcs['std']['email_idx']] == edited_std_responses_df.iloc[row_index, std_idcs['email_idx']]:
                    edited_std_responses_df.iloc[row_index, std_idcs['first_recomm_letter_idx']] = ref_responses_df.iloc[ref_index, ref_idcs['letter_idx']]
                    ref_responses_df.iloc[ref_index, ref_idcs['matched_idx']] = "matched"
                    
                    break
                
            for ref_index in range(len(ref_responses_df)):
                
                if ref_responses_df.iloc[ref_index, ref_idcs['std']['email_idx']] == edited_std_responses_df.iloc[row_index, std_idcs['email_idx']] and edited_std_responses_df.iloc[row_index, std_idcs['first_recomm_letter_idx']] is not None:
                    edited_std_responses_df.iloc[row_index, std_idcs['second_recomm_letter_idx']] = ref_responses_df.iloc[ref_index, ref_idcs['letter_idx']]
                    ref_responses_df.iloc[ref_index, ref_idcs['matched_idx']] = "matched"
        
        # Flag student response if one of or both their references submitted MORE THAN ONE letter
        # And assign the right two letters to that student
        
        elif ref_responses_df[ref_idcs['std']['email_idx']].value_counts()[edited_std_responses_df.iloc[row_index, std_idcs['email_idx']]] > 2:
            
            #edited_std_responses_df.iloc[row_index, std_idcs['flag_idx']] = "flagged"
            edited_std_responses_df = leave_note(edited_std_responses_df, row_index,
                                                 "Some reference/s submitted more than two letters (The last was taken)")
            
            for ref_index in range(len(ref_responses_df)):
                
                if ref_responses_df.iloc[ref_index, ref_idcs['std']['email_idx']] == edited_std_responses_df.iloc[row_index, std_idcs['email_idx']] and ref_responses_df.iloc[ref_index, ref_idcs['flag_idx']] is None:
                    
                    edited_std_responses_df.iloc[row_index, std_idcs['first_recomm_letter_idx']] = ref_responses_df.iloc[ref_index, ref_idcs['letter_idx']]
                    ref_responses_df.iloc[ref_index, ref_idcs['matched_idx']] = "matched"
                    
                    break
                    
            for ref_index in range(len(ref_responses_df)):
                
                if ref_responses_df.iloc[ref_index, ref_idcs['std']['email_idx']] == edited_std_responses_df.iloc[row_index, std_idcs['email_idx']] and edited_std_responses_df.iloc[row_index, std_idcs['first_recomm_letter_idx']] is not None and ref_responses_df.iloc[ref_index, ref_idcs['flag_idx']] is None:
                    
                    edited_std_responses_df.iloc[row_index, std_idcs['second_recomm_letter_idx']] = ref_responses_df.iloc[ref_index, ref_idcs['letter_idx']]
                    ref_responses_df.iloc[ref_index, ref_idcs['matched_idx']] = "matched"
                    
    return edited_std_responses_df, ref_responses_df


def match_refs_based_on_stdn_name(std_responses_df, ref_responses_df):
    """
    Matches references with the student/s they are supporting, and flags student response if they get less than the required
    number of reference letters, and leaves a note.
    
    params :
        std_responses_df : students responses data (DataFrame)
        ref_responses_df : references responses data (DataFrame)
    returns:
        edited_std_responses_df: An edited std_responses_df with answers with unsatisfied conditions for recommendation letters flagged,
        ref_responses_df: The ref_responses_df but with marking the "Matched" column for letters those successfully matched.
    """
     
    ref_responses_df = flag_duplicate_refs(ref_responses_df)
    edited_std_responses_df = std_responses_df.copy()
    
    for row_index in range(len(edited_std_responses_df)):
        
        if edited_std_responses_df.iloc[row_index, std_idcs['first_recomm_letter_idx']] is not None and edited_std_responses_df.iloc[row_index, std_idcs['second_recomm_letter_idx']] is not None:
            continue
            
        try:
            got_one_letter = ref_responses_df[[ref_idcs['std']['firstname_idx'], ref_idcs['std']['lastname_idx']]].value_counts()[tuple(edited_std_responses_df.iloc[row_index, std_idcs['firstname_idx']:std_idcs['lastname_idx'] + 1])] == 1
        
        except KeyError:
            got_one_letter = False
            
        try:
            got_two_letters = ref_responses_df[[ref_idcs['std']['firstname_idx'], ref_idcs['std']['lastname_idx']]].value_counts()[tuple(edited_std_responses_df.iloc[row_index, std_idcs['firstname_idx']:std_idcs['lastname_idx'] + 1])] == 2

        except KeyError:
            got_two_letters = False 
            
        try:
            got_more_than_two_letters = ref_responses_df[[ref_idcs['std']['firstname_idx'], ref_idcs['std']['lastname_idx']]].value_counts()[tuple(edited_std_responses_df.iloc[row_index, std_idcs['firstname_idx']:std_idcs['lastname_idx'] + 1])] > 2
       
        except KeyError:
            got_more_than_two_letters = False 
        
        if not got_one_letter and not got_two_letters and not got_more_than_two_letters:
            got_no_letters = True
            
        else:
            got_no_letters = False
    
        # Flag student response if BOTH of their references didn't submit any letter
          
        if got_no_letters:
            
            edited_std_responses_df.iloc[row_index, std_idcs['flag_idx']] = "flagged"
            edited_std_responses_df = leave_note(edited_std_responses_df, row_index, "Got no recommendation letters")
        
        # Flag student response if ANY of their references didn't submit any letter
        # Assign the one submitted letters to that student
            
        if got_one_letter:
            
            edited_std_responses_df.iloc[row_index, std_idcs['flag_idx']] = "flagged"
            edited_std_responses_df = leave_note(edited_std_responses_df, row_index, "Got only one recommendation letter")
            
            for ref_index in range(len(ref_responses_df)):
                
                if ref_responses_df.iloc[ref_index, [ref_idcs['std']['firstname_idx'], ref_idcs['std']['lastname_idx']]].tolist() == edited_std_responses_df.iloc[row_index, std_idcs['firstname_idx']:std_idcs['lastname_idx'] + 1].tolist():
                    
                    edited_std_responses_df.iloc[row_index, std_idcs['first_recomm_letter_idx']] = ref_responses_df.iloc[ref_index, ref_idcs['letter_idx']]
                    ref_responses_df.iloc[ref_index, ref_idcs['matched_idx']] = "matched"
                    
                    break
        
        # This, from here below, would look much prettier with a while loop!
    
        # If BOTH references subnitted ONLY ONE letter,
        # Assign the right two letters to the specific student
        
        if got_two_letters:
            
            for ref_index in range(len(ref_responses_df)):
                
                if ref_responses_df.iloc[ref_index, [ref_idcs['std']['firstname_idx'], ref_idcs['std']['lastname_idx']]].tolist() == edited_std_responses_df.iloc[row_index, std_idcs['firstname_idx']:std_idcs['lastname_idx'] + 1].tolist():
                    
                    edited_std_responses_df.iloc[row_index, std_idcs['first_recomm_letter_idx']] = ref_responses_df.iloc[ref_index, ref_idcs['letter_idx']]
                    ref_responses_df.iloc[ref_index, ref_idcs['matched_idx']] = "matched"
                    
                    break
                
            for ref_index in range(len(ref_responses_df)):
                
                if ref_responses_df.iloc[ref_index, [ref_idcs['std']['firstname_idx'], ref_idcs['std']['lastname_idx']]].tolist() == edited_std_responses_df.iloc[row_index, std_idcs['firstname_idx']:std_idcs['lastname_idx'] + 1].tolist() and edited_std_responses_df.iloc[row_index, std_idcs['first_recomm_letter_idx']] is not None:
                    
                    edited_std_responses_df.iloc[row_index, std_idcs['second_recomm_letter_idx']] = ref_responses_df.iloc[ref_index, ref_idcs['letter_idx']]
                    ref_responses_df.iloc[ref_index, ref_idcs['matched_idx']] = "matched"
        
        # Check if there are references who submitted MORE THAN ONE letter to the same student
        # And assign the right two letters to that student
        
        if got_more_than_two_letters:
            
            edited_std_responses_df = leave_note(edited_std_responses_df, row_index,
                                                 "Some reference/s submitted more than two letters (The last was taken)")

            for ref_index in range(len(ref_responses_df)):

                if ref_responses_df.iloc[ref_index, [ref_idcs['std']['firstname_idx'], ref_idcs['std']['lastname_idx']]].tolist() == edited_std_responses_df.iloc[row_index, std_idcs['firstname_idx']:std_idcs['lastname_idx'] + 1].tolist() and ref_responses_df.iloc[ref_index, ref_idcs['flag_idx']] is None:

                    edited_std_responses_df.iloc[row_index, std_idcs['first_recomm_letter_idx']] = ref_responses_df.iloc[ref_index, ref_idcs['letter_idx']]
                    ref_responses_df.iloc[ref_index, ref_idcs['matched_idx']] = "matched"

                    break

            for ref_index in range(len(ref_responses_df)):

                if ref_responses_df.iloc[ref_index, [ref_idcs['std']['firstname_idx'], ref_idcs['std']['lastname_idx']]].tolist() == edited_std_responses_df.iloc[row_index, std_idcs['firstname_idx']:std_idcs['lastname_idx'] + 1].tolist() and edited_std_responses_df.iloc[row_index, std_idcs['first_recomm_letter_idx']] is not None and ref_responses_df.iloc[ref_index, ref_idcs['flag_idx']] is None:

                    edited_std_responses_df.iloc[row_index, std_idcs['second_recomm_letter_idx']] = ref_responses_df.iloc[ref_index, ref_idcs['letter_idx']]
                    ref_responses_df.iloc[ref_index, ref_idcs['matched_idx']] = "matched"
                    
    return edited_std_responses_df, ref_responses_df


def match_references(std_responses_df, ref_responses_df):
    
    #edited_std_responses_df, ref_responses_df = match_refs_based_on_stdn_email(std_responses_df, ref_responses_df)
    edited_std_responses_df, ref_responses_df = match_refs_based_on_stdn_name(std_responses_df, ref_responses_df)
    
    return edited_std_responses_df, ref_responses_df

In [761]:
def main(std_responses_df, ref_responses_df):
    
    std_responses_df = column_names_to_indices(std_responses_df, std_questions_dict)
    ref_responses_df = column_names_to_indices(ref_responses_df, ref_questions_dict)
    
    responses_df_flagged_duplicates = flag_duplicates(std_responses_df)
    responses_df_flagged_short = flag_short(responses_df_flagged_duplicates, essay_qs)
    responses_df_flagged_long = flag_long (responses_df_flagged_short, essay_qs)
    
    responses_df_spaces_removed, ref_responses_df_spaces_removed = remove_spaces(responses_df_flagged_long, std_str_qs, ref_responses_df, ref_str_qs)
    
    responses_df_lowercase, ref_responses_df_lowercase = to_lowercase(responses_df_spaces_removed, ref_responses_df_spaces_removed)
    
    responses_df_matched, ref_responses_df_matched = match_references(responses_df_lowercase, ref_responses_df_lowercase)
    ref_responses_df_final = flag_refs_with_no_students(responses_df_matched, ref_responses_df_matched)
    
    responses_df_final, ref_responses_df_final = to_uppercase(responses_df_matched, ref_responses_df_final)
    
    ref_responses_df_unmatched = get_unmatched_letters(responses_df_final, ref_responses_df_final)
    named_ref_responses_df_unmatched = indices_to_column_names(ref_responses_df_unmatched, ref_questions_dict)
    named_ref_responses_df_unmatched.to_excel(RESULTS_FOLDER_NAME + "/unmatched_letters.xlsx")
    named_ref_responses_df_unmatched.to_excel(LETTERS_STATS_FOLDER_NAME + "/unmatched_letters.xlsx")
    
    # Putting back original column names, and saving the Excel file
    named_responses_df_final = indices_to_column_names(responses_df_final, std_questions_dict)
    named_responses_df_final.to_excel(RESULTS_FOLDER_NAME + "/filtered_responses_with_flagged.xlsx")
    
    save_letters_stats(responses_df_final, ref_responses_df_final)
    
    responses_df_final_flagged_removed = remove_flagged(responses_df_final)
    
    # Putting back original column names, and saving the Excel file
    named_responses_df_final_flagged_removed = indices_to_column_names(responses_df_final_flagged_removed, std_questions_dict)
    named_responses_df_final_flagged_removed.to_excel(RESULTS_FOLDER_NAME + "/filtered_responses_with_flagged_removed.xlsx")
    
    # Putting back original column names, and saving the Excel file
    named_ref_responses_df_final = indices_to_column_names(ref_responses_df_final, ref_questions_dict)
    named_ref_responses_df_final.to_excel(RESULTS_FOLDER_NAME + "/ref_responses_with_flagged.xlsx")
    
    return responses_df_final, responses_df_final_flagged_removed, ref_responses_df_final, ref_responses_df_unmatched

In [762]:
responses_df_final, responses_df_final_flagged_removed, ref_responses_df_final, ref_responses_df_unmatched = main(std_raw_responses_df, ref_raw_responses_df)

In [763]:
get_std_summary(responses_df_final)

,1,2,3,28,29,30,31
0,jakindaodhiambo@gmail.com,Jakinda,Oluoch,flagged,Got no recommendation letters,None,None
1,ejirukomaotomewo@gmail.com,Lily,Otomewo,None,None,https://drive.google.com/open?id=1u_zf7D7hyrz7...,https://drive.google.com/open?id=1PVa-LI3hCZXF...
2,mohamedahmed@aims.edu.gh,Mohamedahmed,Mohamed,flagged,Got only one recommendation letter,https://drive.google.com/open?id=1Qy8idpxRI1ss...,None
3,2567491@students.wits.ac.za,Talha,Niazi,flagged,Got only one recommendation letter,https://drive.google.com/open?id=1E67XTrEB9xNQ...,None
4,paul.fadojutimi@wits.ac.za,Paul,Fadojutimi,None,Some reference/s submitted more than two lette...,https://drive.google.com/open?id=1nAmQu9kCGvyB...,https://drive.google.com/open?id=1STRlQ7RH2hR0...
...,...,...,...,...,...,...,...
421,njohjammundih@yahoo.com,Mundih,Njohjam,flagged,Got no recommendation letters,None,None
422,adenijik@babcock.edu.ng,Kehinde,Adeniji,flagged,Got no recommendation letters,None,None
423,danila2001606@gmail.com,Danila,Blagomirov,flagged,Got no recommendation letters,None,None
424,sjohnny@andrew.cmu.edu,Samuelq,Johnny,flagged,Got no recommendation letters,None,None


In [764]:
ref_responses_df_final

,0,1,2,3,4,5,6,7,8,9,10
0,2024-01-16 15:34:32.401,siziwe.gqoba@wits.ac.za,Dr Siziwe Gqoba,Paul,Fadojutimi,paul.fadojutimi@wits.ac.za,Host (Supervisor),https://drive.google.com/open?id=1nAmQu9kCGvyB...,None,None,matched
1,2024-01-16 15:39:28.792,nosipho.moloto@wits.ac.za,Prof. Nosipho Moloto,Paul,Fadojutimi,paul.fadojutimi@wits.ac.za,Host (Supervisor),https://drive.google.com/open?id=1vQZBhTcTUGSH...,flagged,Submitted more than one letter for the same st...,unmatched
2,2024-01-16 16:53:49.704,adeoluwaoa@abuad.edu.ng,"Dr Olusegun Adeoluwa, Afe Babalola University",Lily,Otomewo,ejirukomaotomewo@gmail.com,Student and co-worker,https://drive.google.com/open?id=1u_zf7D7hyrz7...,None,None,matched
3,2024-01-16 17:03:11.842,pietrocap@gmail.com,"Professor Pietro Caputo, Roma Tre University",Mohamedahmed,Mirghanihassan,mohamedahmed@aims.edu.gh,Teacher,https://drive.google.com/open?id=1NKmQAGlOyDXe...,None,None,unmatched
4,2024-01-16 19:04:15.104,andrew.mwaura@uonbi.ac.ke,"Prof. Andrew Mwaura Kahonge, The University of...",Symprose,Jangaya,purityjangaya@students.uonbi.ac.ke,I am her lecturer,https://drive.google.com/open?id=118B4SZ6028nI...,None,None,unmatched
...,...,...,...,...,...,...,...,...,...,...,...
239,2024-02-02 21:14:46.077,jc5737@cumc.columbia.edu,"Josue Curto Navarro, PhD. Postdoctoral Researc...",Abdulahi,Amusa,amusatomisin65@gmail.com,We collaborated on a Project to build a ML mod...,https://drive.google.com/open?id=16xxGJcu6wCpX...,None,"Their student didn't submit an application, or...",unmatched
240,2024-02-03 04:25:55.078,isaini@luanar.ac.mw,"Agronomy Technician , Ian Madalitso Saini , Li...",Bond,Chiwewe,bond@amityonline.com,Project counterpart,https://drive.google.com/open?id=1kGLkmxu70uDp...,None,"Their student didn't submit an application, or...",matched
241,2024-02-03 05:20:05.640,emmanuel.e@serenityinmind.co.uk,Mr Emmanuel Prince Ehimhen,Fimbarr,Okonkwo,ekenedilichukwueo366@gmail.com,Mentor,https://drive.google.com/open?id=1ogSDVhXQ19Zk...,None,None,unmatched
242,2024-02-03 07:22:47.183,gerigora@gmail.com,Lecturer,Mariegrace,Niyongira,niyoracekamali1@gmail.com,My student,https://drive.google.com/open?id=1TnBjNS-4wn8O...,None,"Their student didn't submit an application, or...",unmatched


In [765]:
len(ref_responses_df_unmatched)

92

In [766]:
get_std_summary(responses_df_final_flagged_removed)

,1,2,3,28,29,30,31
1,ejirukomaotomewo@gmail.com,Lily,Otomewo,None,None,https://drive.google.com/open?id=1u_zf7D7hyrz7...,https://drive.google.com/open?id=1PVa-LI3hCZXF...
4,paul.fadojutimi@wits.ac.za,Paul,Fadojutimi,None,Some reference/s submitted more than two lette...,https://drive.google.com/open?id=1nAmQu9kCGvyB...,https://drive.google.com/open?id=1STRlQ7RH2hR0...
10,samuel.ocen@students.uonbi.ac.ug,Samuel,Ocen,None,None,https://drive.google.com/open?id=1xnt12ISA9j5y...,https://drive.google.com/open?id=1sjZ1XnPtRuys...
21,gampascaljoel@gmail.com,Gambo,Suleiman,None,None,https://drive.google.com/open?id=1yNxKTiOduEkx...,https://drive.google.com/open?id=1Yu0T4jqbDHEs...
34,sbganawakili@gmail.com,Shuaibu,Babagana,None,None,https://drive.google.com/open?id=1DeUnRUYW2m8a...,https://drive.google.com/open?id=1XL9aDqKdv_fo...
44,abdullmohedu@gmail.com,Abdullahi,Mohammed,None,None,https://drive.google.com/open?id=1vp0j3II9oCki...,https://drive.google.com/open?id=1pmBh1O1HF8b5...
48,merna.amr@aucegypt.edu,Merna,Saleh,None,None,https://drive.google.com/open?id=1RGmPfnU3D_HT...,https://drive.google.com/open?id=1bp8nrzQyFuQU...
76,louisa.ayamga@acity.edu.gh,Louisa,Ayamga,None,None,https://drive.google.com/open?id=1P1EgdsRfedXC...,https://drive.google.com/open?id=1vfDfdlCznzbY...
79,mustaphasalihu6773@gmail.com,Mustapha,Salihu,None,None,https://drive.google.com/open?id=1pZ62Z8qZqs4p...,https://drive.google.com/open?id=1OIaBcmDj7UMO...
101,omo_rich@yahoo.com,Richard,Oveh,None,None,https://drive.google.com/open?id=1bj-Dh5DxMQkD...,https://drive.google.com/open?id=1ksoAQ9oTWJQo...


In [767]:
len(responses_df_final)

426

In [768]:
len(got_no_enough_letters(responses_df_final))

395

In [769]:
len(got_enough_letters(responses_df_final))

31

In [770]:
get_letters_stats(ref_responses_df_final)

,5,# letters
0,1363409@students.wits.ac.za,1
1,2095007@students.wits.ac.za,1
2,2232089@students.wits.ac.za,1
3,2305062@students.wits.ac.za,1
4,2567491@students.wits.ac.za,1
...,...,...
180,wandaguela53@gmail.com,2
181,wawerualbert58@gmail.com,1
182,yahayaibrahimu88@gmail.com,1
183,yaliyugk@gmail.com,2
